# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [12]:
# import required libraries

import pandas as pd
import numpy as np
import os
import statistics as stats

import warnings
warnings.filterwarnings('ignore')

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [13]:
# your code here
orders = pd.read_csv("Orders.csv")

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [14]:
# your code here
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397924 entries, 0 to 397923
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    397924 non-null  int64  
 1   InvoiceNo     397924 non-null  int64  
 2   StockCode     397924 non-null  object 
 3   year          397924 non-null  int64  
 4   month         397924 non-null  int64  
 5   day           397924 non-null  int64  
 6   hour          397924 non-null  int64  
 7   Description   397924 non-null  object 
 8   Quantity      397924 non-null  int64  
 9   InvoiceDate   397924 non-null  object 
 10  UnitPrice     397924 non-null  float64
 11  CustomerID    397924 non-null  int64  
 12  Country       397924 non-null  object 
 13  amount_spent  397924 non-null  float64
dtypes: float64(2), int64(8), object(4)
memory usage: 42.5+ MB


In [15]:
#1. Agrupamos con groupby los cliente con el gasto:

gasto_cliente = orders.groupby("CustomerID")["amount_spent"].sum()
gasto_cliente

CustomerID
12346    77183.60
12347     4310.00
12348     1797.24
12349     1757.55
12350      334.40
           ...   
18280      180.60
18281       80.82
18282      178.05
18283     2094.88
18287     1837.28
Name: amount_spent, Length: 4339, dtype: float64

In [16]:
#2. 
stats = orders.describe().T
stats["IQR"] = stats["75%"] - stats["25%"]
stats

,count,mean,std,min,25%,50%,75%,max,IQR
Unnamed: 0,397924.0,278465.221859,152771.368303,0.0,148333.75,284907.50,410079.25,541908.00,261745.50
InvoiceNo,397924.0,560617.126645,13106.167695,536365.0,549234.00,561893.00,572090.00,581587.00,22856.00
year,397924.0,2010.934259,0.247829,2010.0,2011.00,2011.00,2011.00,2011.00,0.00
month,397924.0,7.612537,3.416527,1.0,5.00,8.00,11.00,12.00,6.00
day,397924.0,3.614555,1.928274,1.0,2.00,3.00,5.00,7.00,3.00
hour,397924.0,12.728247,2.273535,6.0,11.00,13.00,14.00,20.00,3.00
Quantity,397924.0,13.021823,180.420210,1.0,2.00,6.00,12.00,80995.00,10.00
UnitPrice,397924.0,3.116174,22.096788,0.0,1.25,1.95,3.75,8142.75,2.50
CustomerID,397924.0,15294.315171,1713.169877,12346.0,13969.00,15159.00,16795.00,18287.00,2826.00
amount_spent,397924.0,22.394749,309.055588,0.0,4.68,11.80,19.80,168469.60,15.12


In [17]:
percn_75 = np.percentile(gasto_cliente, 75)  
print(percn_75)
percn_95 = np.percentile(gasto_cliente, 95)
print(percn_95) 

1661.64
5840.181999999982


In [18]:
#3.
orders["Customer_Label"] = "Normal"
vip = gasto_cliente > percn_95
vip_index = gasto_cliente[vip].index
orders.loc[orders["CustomerID"].isin(vip_index), "Customer_Label"] = "VIP"

In [19]:
preferred_customers = (gasto_cliente >= percn_75) & (gasto_cliente <= percn_95)
preferred_customers_index = gasto_cliente[preferred_customers].index
orders.loc[orders["CustomerID"].isin(preferred_customers_index), "Customer_Label"] = "Preferred"
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Customer_Label
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [20]:
# your code here:

vip_customers_df = orders[orders["Customer_Label"] == "VIP"]
country_vip = vip_customers_df["Country"].value_counts()
most_vip_country = country_vip.idxmax()
most_vip_country

'United Kingdom'

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [21]:
# your code here:
vip_preferred_df = orders[orders["Customer_Label"].isin(["VIP", "Preferred"])]
vip_preferred_country = vip_preferred_df["Country"].value_counts()

In [22]:
vip_preferred_country_sorted = vip_preferred_country.sort_values(ascending=False)
vip_preferred_country_sorted.head(10)

Country
United Kingdom    221635
Germany             7349
EIRE                7238
France              6301
Netherlands         2080
Spain               1569
Belgium             1557
Switzerland         1370
Portugal            1093
Norway              1028
Name: count, dtype: int64

In [23]:
vip_preferred = vip_preferred_country_sorted.index[0]
vip_preferred

'United Kingdom'